In [1]:
from pyoculus.problems import AnalyticCylindricalBfield
from pyoculus.solvers import PoincarePlot, FixedPoint, Manifold
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
separatrix = {"type": "circular-current-loop", "amplitude": -10, "R": 6, "Z": -5.5}
maxwellboltzmann = {"m": 18, "n": -3, "d": np.sqrt(2), "type": "maxwell-boltzmann", "amplitude": 1e-2}

# Creating the pyoculus problem object, adding the perturbation here use the R, Z provided as center point
pyoproblem = AnalyticCylindricalBfield.without_axis(
    6,
    0,
    0.91,
    0.6,
    perturbations_args=[separatrix],
    Rbegin=1,
    Rend=8,
    niter=800,
    guess=[6.41, -0.7],
    tol=1e-9,
)

# # Adding perturbation after the object is created uses the found axis as center point
pyoproblem.add_perturbation(maxwellboltzmann)

### Finding the X-point
print("\nFinding the X-point\n")

# set up the integrator for the FixedPoint
iparams = dict()
iparams["rtol"] = 1e-13

pparams = dict()
pparams["nrestart"] = 0
pparams["niter"] = 300

# set up the FixedPoint object
fixedpoint = FixedPoint(pyoproblem, pparams, integrator_params=iparams)

# find the X-point
guess = [6.21560891, -4.46981856]
print(f"Initial guess: {guess}")

fixedpoint.compute(guess=guess, pp=0, qq=1, sbegin=4, send=9, tol=1e-10)

if fixedpoint.successful:
    results = [list(p) for p in zip(fixedpoint.x, fixedpoint.y, fixedpoint.z)]
else:
    raise ValueError("X-point not found")

# Set up the manifold
iparams = dict()
iparams["rtol"] = 1e-13
manifold = Manifold(fixedpoint, pyoproblem, integrator_params=iparams)

# Choose the tangles to work with
manifold.choose(0, 0)

0 - dr : 0.010422282084043258
1 - RZ : [ 6.41409394 -0.69368043]
1 - dr : 6.293042617255254e-06
2 - RZ : [ 6.41409781 -0.69367863]
2 - dr : 9.776471937205409e-12
0 - dr : 9.776471937205409e-12

Finding the X-point

Initial guess: [6.21560891, -4.46981856]
0 - [DeltaR, DeltaZ] : [0.36802375 0.29365015] - dtheta : 0.10115931465688033
0 - [StepR, StepZ]: [-0.02359085 -0.02028676]
1 - RZ : [ 6.19201806 -4.49010532] - rhotheta : [ 3.80291665 -1.62922679]
1 - [DeltaR, DeltaZ] : [-0.1610547  -0.17320114] - dtheta : 0.037787972266624115
1 - [StepR, StepZ]: [0.00558637 0.00257023]
2 - RZ : [ 6.19760443 -4.48753509] - rhotheta : [ 3.80002846 -1.62779871]
2 - [DeltaR, DeltaZ] : [-0.01703985 -0.02899794] - dtheta : 0.004010505876924864
2 - [StepR, StepZ]: [ 0.00597171 -0.0082389 ]
3 - RZ : [ 6.20357614 -4.49577399] - rhotheta : [ 3.80791918 -1.62610976]
3 - [DeltaR, DeltaZ] : [0.0003958  0.00018181] - dtheta : 0.00010114859451793023
3 - [StepR, StepZ]: [ 5.73037418e-05 -1.31785770e-04]
4 - RZ : [ 

In [ ]:
fig = pickle.load(open("manifold_05070828.pkl", "rb"))
ax = fig.gca()

In [ ]:
eps_s_1, eps_u_1 = manifold.find_homoclinic(5.894806378673835e-07, 1.6269136139414627e-06)
bounds_1 = manifold.find_bounds(eps_s_1, eps_u_1)

In [ ]:
manifold.clinics

In [ ]:
bounds_i = np.array(bounds_1)
bounds_i[0][0] = manifold.clinics[-1][1]
bounds_i[1][1] = manifold.clinics[-1][2]
bounds_i = (tuple(bounds_i[0]), tuple(bounds_i[1]))

In [ ]:
bounds_1

In [ ]:
bounds_i

In [ ]:
# i = 1
fund = manifold.fundamental_segment
# guess_i = [bounds_1[0][1]*np.power(manifold.lambda_s, 5/6), bounds_1[1][0]*np.power(manifold.lambda_u, 5/6)]
guess_i = [bounds_1[0][1]*np.power(manifold.lambda_s, 5/6), bounds_1[1][0]*np.power(manifold.lambda_u, 5/6)]
print(f"initial guess: {guess_i}")

In [ ]:
manifold.find_homoclinic(5.894806378673835e-07, 1.6269136139414627e-06, n_s=7, n_u=6)
manifold.find_homoclinic(9.537056471124823e-06, 2.7840434321883966e-06, n_s=7, n_u=5)
manifold.find_homoclinic(5.512302400879586e-06, 4.975036258371061e-06, n_s=7, n_u=5)
manifold.find_homoclinic(3.1187737295664092e-06, 8.513476637442009e-06, n_s=7, n_u=5)
manifold.find_homoclinic(1.8025954174027533e-06, 1.5213519614175902e-05, n_s=7, n_u=5)
manifold.find_homoclinic(1.0198530741559527e-06, 2.603439823207209e-05, n_s=7, n_u=5)
manifold.order()

In [3]:
# manifold.find_homoclinic(1.2031633045398876e-06, 1.4761625196001464e-06, n_s=7, n_u=6)
manifold.find_clinics(n_points=6)
manifold.order()

2024-05-22 13:28:02 [info     ] Search for minimum of the linear error succeeded, epsilon = 2.05960e-07
2024-05-22 13:28:02 [info     ] Search for minimum of the linear error succeeded, epsilon = 3.03587e-07
2024-05-22 13:28:02 [debug    ] Guess - 2.8417669398141114e-06, 4.188790607008011e-06
2024-05-22 13:28:02 [debug    ] Bounds - ((2.0595967129152613e-07, 5.889493550919749e-06), (3.03586922011375e-07, 8.681168136027398e-06))
2024-05-22 13:28:02 [debug    ] n_s, n_u - 7, 6
2024-05-22 13:28:02 [debug    ] Inside : (2.8417669398141114e-06, 4.188790607008009e-06)
2024-05-22 13:28:03 [debug    ] Returns - (array([8.49316493, 0.76715983]), array([5.35459693, 1.96370928]), array([ 3.138568  , -1.19654945]))
2024-05-22 13:28:03 [debug    ] Inside : (2.8417669398141114e-06, 4.188790607008009e-06)
2024-05-22 13:28:03 [debug    ] Returns - (array([8.49316493, 0.76715983]), array([5.35459693, 1.96370928]), array([ 3.138568  , -1.19654945]))
2024-05-22 13:28:03 [debug    ] Inside : (2.8417669398

In [ ]:
eps_s_i, eps_u_i = manifold.find_homoclinic(*guess_i, bounds = bounds_i, n_s=7, n_u=5, options={"factor": 0.1})

In [ ]:
manifold.clinics

In [ ]:
fig = pickle.load(open("manifold_05070828.pkl", "rb"))
ax = fig.gca()

In [ ]:
eps_s_i, eps_u_i = manifold.clinics[0][1:3]
# eps_s_i, eps_u_i = bounds_1[0][1], bounds_1[1][1]

n_s, n_u = 6, 6
hs_i = manifold.integrate(manifold.rfp_s + eps_s_i * manifold.vector_s, n_s, -1)[:,-1]
hu_i = manifold.integrate(manifold.rfp_u + eps_u_i * manifold.vector_u, n_u, 1)[:,-1]
ax.scatter(*hs_i, marker='+', color="purple", zorder=10)
ax.scatter(*hu_i, marker='x', color="blue", zorder=10)

In [ ]:
marker = ["o", "s", "*", "^", "v", "<", ">"]

for i in range(4):
    rs, ru = manifold.history[i][:2]
    ax.scatter(*rs, c='purple', marker=marker[i], zorder=10)
    ax.scatter(*ru, c='blue', marker=marker[i], zorder=10)

In [ ]:
manifold.resonance_area()

In [ ]:
manifold.areas

In [ ]:
manifold.errors

In [ ]:
np.array([0,1,2])[3]

In [ ]:
manifold.areas

In [ ]:
ax.set_xlim(3.9, 7)
ax.set_ylim(-0.3, 2.2)
# ax.set_xlim(3.9, 4.3)
# ax.set_ylim(-0.1, 0.6)
# ax.set_xlim(4, 4.05)
# ax.set_ylim(0.45, 0.51)
fig

In [ ]:
marker = ["+", "o", "s", "p", "P", "*", "X", "D", "d", "^", "v", "<", ">"]
for i, clinic in enumerate(manifold.clinics):
    eps_s_i, eps_u_i = clinic[1:3]

    n_s, n_u = 8, 8
    
    hs_i = manifold.integrate(manifold.rfp_s + eps_s_i * manifold.vector_s, n_s, -1)
    hu_i = manifold.integrate(manifold.rfp_u + eps_u_i * manifold.vector_u, n_u, 1)
    ax.scatter(hs_i[0,:], hs_i[1,:], marker=marker[i], color="purple", zorder=10)
    ax.scatter(hu_i[0,:], hu_i[1,:], marker=marker[i], color="blue", zorder=10)

In [ ]:
# ax.set_xlim(6, 8)
# ax.set_ylim(-5, -3.5)
# ax.set_xlim(6, 9)
# ax.set_ylim(-5, -1)
fig

In [ ]:
ax.set_xlim(6.42, 6.555)
ax.set_ylim(-4.32, -4.2)

ax.scatter(6.49, -4.26)

fig

In [ ]:
pyoproblem.B([6.49, 0., -4.26])

In [ ]:
0.01 * 0.1 * pyoproblem.B([6.49, 0., -4.26])[1] * 6.49

In [ ]:
manifold.order()

In [ ]:
manifold.clinics

In [ ]:
def resonance_area(self, n_f = None, n_b = None, n_transit=3):
    # considering the >_u ordering of the clinic points

    potential_integrations = []
    endpoints = []

    # Potential integration
    for clinic in self.clinics:

        n_tmp_f, n_tmp_b = 1, 1

        # Forward integration
        rze_forward = clinic[-2]
        intA_forward = []
        while n_tmp_f <= n_f:
            rze_end, area_tmp = self.integrate_single(
                rze_forward, 1, direction=1, ret_jacobian=False, integrate_A=True
            )

            if n_tmp_f > n_transit and np.linalg.norm(
                rze_end - self.rfp_s
            ) > np.linalg.norm(rze_forward - self.rfp_s):
                print("Forward integration goes beyond the saddle point.")
                print(
                    f"rfp_s: {self.rfp_s}, rze_end: {rze_end}, rze_forward: {rze_forward}"
                )
                n_tmp_f -= 1
                break

            rze_forward = rze_end
            intA_forward.append(area_tmp)
            n_tmp_f += 1

        print(f"Forward integration completed with area {intA_forward}")

        # Backward integration
        # taking the point found from unstable manifold to go back to the fixedpoint
        rze_backward = clinic[-1]
        intA_backward = []
        while n_tmp_b <= n_b:
            rze_end, area_tmp = self.integrate_single(
                rze_backward, 1, direction=-1, ret_jacobian=False, integrate_A=True
            )

            if n_tmp_b > n_transit and np.linalg.norm(
                rze_end - self.rfp_u
            ) > np.linalg.norm(rze_backward - self.rfp_u):
                print("Backward integration goes beyond the saddle point.")
                print(
                    f"rfp_u: {self.rfp_u}, rze_end: {rze_end}, rze_forward: {rze_backward}"
                )
                n_tmp_b -= 1
                break

            rze_backward = rze_end
            intA_backward.append(area_tmp)
            n_tmp_b += 1

        print(f"Backward integration completed with area {intA_backward}")

        potential_integrations.append([np.array(intA_forward), np.array(intA_backward)])
        endpoints.append([rze_forward, rze_backward])

    # Area calculation
    areas = []
    for intA_h, intA_m in zip(potential_integrations, [potential_integrations[i] for i in np.roll(np.arange(len(potential_integrations), dtype=int), -1)]):
        n_fwd = min(intA_h[0].size, intA_m[0].size)
        n_bwd = min(intA_h[1].size, intA_m[1].size)

        intm = intA_m[0][:n_fwd].sum()-intA_m[1][:n_bwd].sum()
        inth = intA_h[0][:n_fwd].sum()-intA_h[1][:n_bwd].sum()

        areas.append(intm - inth)

    areas = np.array(areas)
    
    self.areas = areas

    return areas, potential_integrations, endpoints


In [ ]:
res = resonance_area(manifold, 10, 10, 2)

In [ ]:
res[1][i]

In [ ]:
areas

In [ ]:
manifold.areas

In [ ]:
areas[areas > 0].sum()

In [ ]:
areas[areas < 0].sum()

In [ ]:
areas.sum()

In [ ]:
np.roll(np.array([1,2,3,4]),-1)

In [ ]:
n_bwd

In [ ]:
re

In [ ]:
manifold.rfp_u

In [ ]:
manifold.integrate_single(re[0][0], 1, -1, ret_jacobian = False, integrate_A = True)

In [ ]:
manifold.integrate_single(re[0][1], 1, 1, ret_jacobian = False, integrate_A = True)

In [ ]:
manifold.integrate_single(manifold.rfp_u, 1, -1, ret_jacobian = False, integrate_A = True)

In [ ]:
intAs

In [ ]:
intAs[-1] - intAs[0]

In [ ]:
areas = [intAs[i+1]-intAs[i] for i in range(5)]
areas.append(intAs[-1] - intAs[0])
areas

In [ ]:
int_rfp = manifold.integrate_single(manifold.rfp_u, 1, -1, ret_jacobian = False, integrate_A = True)[1]

In [ ]:
areas[0] = areas[0]+int_rfp

In [ ]:
areas[-2] = -(areas[-2]-int_rfp)

In [ ]:
areas[-1] *= -1

In [ ]:
areas

In [ ]:
np.sum(areas)

In [ ]:
np.sum(areas[::2])

In [ ]:
n_s, n_u = 5, 6
def evolution(self, eps):
    print(eps)
    eps_s, eps_u = eps
    r_s = self.rfp_s + eps_s * self.vector_s
    r_u = self.rfp_u + eps_u * self.vector_u

    try:
        r_s_evolved = self.integrate_single(r_s, n_s, -1, ret_jacobian = False)
    except Exception as e:
        print(f"Error in stable manifold integration : {e}")

    try:
        r_u_evolved = self.integrate_single(r_u, n_u, 1, ret_jacobian = False)
    except Exception as e:
        print(f"Error in unstable manifold integration : {e}")

    return r_s_evolved - r_u_evolved

In [ ]:
eps_s = np.logspace(np.log(bounds_1[0][0]) / np.log(manifold.lambda_s), 
            np.log(bounds_1[0][1]) / np.log(manifold.lambda_s),
            40, base=manifold.lambda_s)

eps_u = np.logspace(np.log(bounds_1[1][0]) / np.log(manifold.lambda_u), 
            np.log(bounds_1[1][1]) / np.log(manifold.lambda_u),
            40, base=manifold.lambda_u)

es, eu = np.meshgrid(eps_s, eps_u)

In [ ]:
E = np.array([evolution(manifold, [S, U]) for S, U in zip(es.flatten(), eu.flatten())]).reshape(es.shape + (2,))

In [ ]:
np.linalg.norm(E[3,2,:])

In [ ]:
N[3,2]

In [ ]:
N = np.linalg.norm(E, axis=-1)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(es, eu, N, cmap='viridis')

plt.show()

In [ ]:
np.save("datas/0002-stables_eps",es)
np.save("datas/0002-unstables_eps", eu)
np.save("datas/0002-elevation", E)
np.save("datas/0002-norm", N)

In [ ]:
eps_s_lb = manifold.find_epsilon(manifold.rfp_s, manifold.vector_s, direction=-1)
eps_u_lb = manifold.find_epsilon(manifold.rfp_u, manifold.vector_u)

bounds = manifold.find_bounds(eps_s_lb, eps_u_lb)

n_s, n_u = manifold.find_N(eps_s_lb, eps_u_lb)
print(n_s, n_u)

In [ ]:
space_u = np.logspace(
            np.log(bounds[1][0]) / np.log(manifold.lambda_u),
            np.log(bounds[1][1]) / np.log(manifold.lambda_u),
            20,
            base=manifold.lambda_u,
        )

space_s = np.logspace(
            np.log(bounds[0][0]) / np.log(manifold.lambda_s),
            np.log(bounds[0][1]) / np.log(manifold.lambda_s),
            20,
            base=manifold.lambda_s,
        )

In [ ]:
n_s, n_u = 6, 7
def evolution(self, eps):
    print(eps)
    eps_s, eps_u = eps
    r_s = self.rfp_s + eps_s * self.vector_s
    r_u = self.rfp_u + eps_u * self.vector_u

    try:
        r_s_evolved = self.integrate_single(r_s, n_s, -1, ret_jacobian = False)
    except Exception as e:
        print(f"Error in stable manifold integration : {e}")

    try:
        r_u_evolved = self.integrate_single(r_u, n_u, 1, ret_jacobian = False)
    except Exception as e:
        print(f"Error in unstable manifold integration : {e}")

    return r_s_evolved - r_u_evolved

es, eu = np.meshgrid(space_s, space_u)
E = np.array([evolution(manifold, [S, U]) for S, U in zip(es.flatten(), eu.flatten())]).reshape(es.shape + (2,))

In [ ]:
N = np.linalg.norm(E, axis=2)

In [ ]:
%matplotlib ipympl

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()

ax = fig.add_subplot()
contour = ax.contourf(es, eu, N)
cbar = plt.colorbar(contour)

# ax = fig.add_subplot(111, projection='3d')
# ax.plot_surface(es, eu, N, cmap='viridis')

plt.show()